In [500]:
from os import listdir
from os.path import isfile, join
# from shapely.geometry import *
# from shapely.wkt import loads
# from shapely import LineString, MultiPoint
# from shapely.ops import nearest_points, split, snap, unary_union

import contextily as cx
import earthaccess
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import os
import pandas as pd
import xarray as xr
import shapely

### Read in and prep NHD

In [684]:
# Get slurm job index
i = 4 # HUC4 0108

In [676]:
def readNHD(index):
    ## Set-up
    mdata_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/data/'
# NEED WITH WATERBODY FOR CUSHMAN!!!!
    prep_path = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC_data/NHD_prepped_with_waterbody/'

    # Define dtypes for lookup tables to preserve leading zeros
    dtype_dic= {'HUC4': str, 'HUC2': str, 'toBasin': str, 'level': str}
    # Read in HUC lookup table
    lookup = pd.read_csv(os.path.join(mdata_path, 'HUC4_lookup_no_great_lakes.csv'), dtype=dtype_dic)

    # Get current HUC2 and HUC4 IDs
    hu2 = 'HUC2_' + lookup.loc[index,'HUC4'][0:2]
    hu4 = 'NHDPLUS_H_' + lookup.loc[index,'HUC4'] + '_HU4_GDB'
    
    # Set data filepath
# NEED WITH WATERBODY FOR CUSHMAN!!!!!
    file_path = os.path.join(prep_path, hu2, hu4 + '_prepped_with_waterbody.gpkg')

    ## Read in prepped NHD flowlines
    features = ['NHDPlusID', 'GNIS_Name', 'LengthKM', 'WidthM', 'Bin', 'geometry']
    basin = gpd.read_file(filename=file_path, columns=features, engine='pyogrio')

    # Drop reaches that are shorter than their width
    basin = basin[basin['LengthKM']*1000 > basin['WidthM']]
    
    # Make geometry 2D LineStrings
    basin['geometry'] = basin.geometry.explode().force_2d()
    return basin

In [685]:
basin = readNHD(index=i)

In [686]:
basin

,Bin,GNIS_Name,LengthKM,NHDPlusID,WidthM,geometry
0,"(0, 10]",None,0.359000,1.000090e+13,2.503269,"LINESTRING (-8132916.01 5289901.543, -8132916...."
1,"(0, 10]",None,0.604000,1.000090e+13,1.535643,"LINESTRING (-8061020.932 5374240.653, -8061022..."
2,"(0, 10]",None,0.317466,1.000090e+13,3.093820,"LINESTRING (-8083828.358 5473168.178, -8083828..."
3,"(0, 10]",None,0.011722,1.000090e+13,2.711831,"LINESTRING (-8018627.084 5542898.946, -8018625..."
4,"(0, 10]",None,0.073721,1.000090e+13,3.184195,"LINESTRING (-8055402.943 5462406.454, -8055377..."
...,...,...,...,...,...,...
96139,"(10, 20]",Stalbird Brook,0.194641,1.000090e+13,16.907900,"LINESTRING (-7960386.79 5534231.437, -7960400...."
96140,"(0, 10]",None,2.661000,1.000090e+13,2.975611,"LINESTRING (-7951969.936 5527770.608, -7951985..."
96141,"(0, 10]",None,1.603000,1.000090e+13,2.463438,"LINESTRING (-7945927.353 5522423.902, -7945917..."
96142,"(0, 10]",None,0.464000,1.000090e+13,1.351103,"LINESTRING (-7930094.548 5646183.585, -7930071..."


#### Finding tiles with geolocation_qual < 4096

##### Download (one-time)

In [132]:
# 341_229R
pixc_results = earthaccess.search_data(short_name = 'SWOT_L2_HR_PIXC_2.0',
                                       temporal = ('2024-03-01 00:00:00', '2024-08-01 23:59:59'),
                                       granule_name = '*_341_229R_*'
                                       # granule_name='*_298_080L_*'
                                      ) # Fort River/Cushman Brook

In [133]:
pixc_results

[Collection: {'Version': '2.0', 'ShortName': 'SWOT_L2_HR_PIXC_2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -72.90260026489858, 'SouthBoundingCoordinate': 41.913629371218626, 'EastBoundingCoordinate': -71.90972118240842, 'NorthBoundingCoordinate': 42.68170665307868}]}, 'Track': {'Cycle': 13, 'Passes': [{'Pass': 341, 'Tiles': ['229R']}]}}}
 Temporal coverage: {'RangeDateTime': {'EndingDateTime': '2024-04-08T18:45:00.181Z', 'BeginningDateTime': '2024-04-08T18:44:49.087Z'}}
 Size(MB): 705.8590621948242
 Data: ['https://archive.swot.podaac.earthdata.nasa.gov/podaac-swot-ops-cumulus-protected/SWOT_L2_HR_PIXC_2.0/SWOT_L2_HR_PIXC_013_341_229R_20240408T184449_20240408T184500_PIC0_01.nc'],
 Collection: {'Version': '2.0', 'ShortName': 'SWOT_L2_HR_PIXC_2.0'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'BoundingRectangles': [{'WestBoundingCoordinate': -72.90275315539066, 'SouthBoundingCoordinate': 41.9118320531

In [149]:
# download
earthaccess.download(pixc_results[-3], '../data/small_rivers/mar_2024_ver_c/')

QUEUEING TASKS | : 100%|██████████| 1/1 [00:00<00:00, 1352.56it/s]
PROCESSING TASKS | : 100%|██████████| 1/1 [00:18<00:00, 18.76s/it]
COLLECTING RESULTS | : 100%|██████████| 1/1 [00:00<00:00, 15196.75it/s]


['../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_016_341_229R_20240610T090004_20240610T090015_PIC0_01.nc']

In [150]:
# Paths to PIXC files
leaf_on = ['../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_018_298_080L_20240720T131306_20240720T131317_PIC0_03.nc',
           '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_016_341_229R_20240610T090004_20240610T090015_PIC0_01.nc',
           '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_017_341_229R_20240701T054507_20240701T054518_PIC0_01.nc',
           '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_018_341_229R_20240722T023012_20240722T023023_PIC0_01.nc']
# I checked, and there were still no leaves in late April 2024
leaf_off = ['../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_012_298_080L_20240317T084237_20240317T084248_PIC0_01.nc',
            '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_013_298_080L_20240407T052741_20240407T052752_PIC0_01.nc',
            '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_014_298_080L_20240428T021247_20240428T021258_PIC0_01.nc',
            '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_013_341_229R_20240408T184449_20240408T184500_PIC0_01.nc',
            '../data/small_rivers/mar_2024_ver_c/SWOT_L2_HR_PIXC_014_341_229R_20240429T152954_20240429T153005_PIC0_01.nc']

##### Read in and look at flags

In [161]:
# Read in xarray
ds_PIXC = xr.open_mfdataset(paths=leaf_off[4], group = 'pixel_cloud', engine='h5netcdf')

In [180]:
ds_PIXC

<xarray.Dataset> Size: 2GB
Dimensions:                                (points: 6609347, complex_depth: 2,
                                            num_pixc_lines: 3271)
Coordinates:
    latitude                               (points) float64 53MB dask.array<chunksize=(508412,), meta=np.ndarray>
    longitude                              (points) float64 53MB dask.array<chunksize=(508412,), meta=np.ndarray>
Dimensions without coordinates: points, complex_depth, num_pixc_lines
Data variables: (12/61)
    azimuth_index                          (points) float64 53MB dask.array<chunksize=(944193,), meta=np.ndarray>
    range_index                            (points) float64 53MB dask.array<chunksize=(944193,), meta=np.ndarray>
    interferogram                          (points, complex_depth) float32 53MB dask.array<chunksize=(1652337, 1), meta=np.ndarray>
    power_plus_y                           (points) float32 26MB dask.array<chunksize=(944193,), meta=np.ndarray>
    power_minus_y                          (points) float32 26MB dask.array<chunksize=(944193,), meta=np.ndarray>
    coherent_power                         (points) float32 26MB dask.array<chunksize=(944193,), meta=np.ndarray>
    ...                                     ...
    pixc_line_qual                         (num_pixc_lines) float64 26kB dask.array<chunksize=(3271,), meta=np.ndarray>
    pixc_line_to_tvp                       (num_pixc_lines) float32 13kB dask.array<chunksize=(3271,), meta=np.ndarray>
    data_window_first_valid                (num_pixc_lines) float64 26kB dask.array<chunksize=(3271,), meta=np.ndarray>
    data_window_last_valid                 (num_pixc_lines) float64 26kB dask.array<chunksize=(3271,), meta=np.ndarray>
    data_window_first_cross_track          (num_pixc_lines) float32 13kB dask.array<chunksize=(3271,), meta=np.ndarray>
    data_window_last_cross_track           (num_pixc_lines) float32 13kB dask.array<chunksize=(3271,), meta=np.ndarray>
Attributes:
    description:                 cloud of geolocated interferogram pixels
    interferogram_size_azimuth:  3271
    interferogram_size_range:    4726
    looks_to_efflooks:           1.5470152426164074
    num_azimuth_looks:           7.0
    azimuth_offset:              5

In [163]:
counts = pd.DataFrame(ds_PIXC.geolocation_qual).value_counts().sort_index()
counts

0       
0.0          362643
1.0           19510
2.0              75
3.0              12
4.0         5292836
5.0          719368
6.0            1111
7.0             437
4096.0          132
4097.0           13
4098.0            1
4100.0        44754
4101.0         6101
4102.0           24
4103.0            5
524288.0       3893
524289.0        357
524290.0          2
524292.0     149645
524293.0       6670
524294.0        317
524295.0         16
528384.0          3
528385.0          3
528388.0       1338
528389.0         80
528390.0          1
Name: count, dtype: int64

In [85]:
counts.sum()

np.int64(4776422)

#### Read in Pixel Cloud

In [650]:
def bitwiseMask(ds):
    # Fow now, eliminate the really bad stuff
    mask = np.where(np.logical_and(ds.classification > 1, ds.geolocation_qual < 2**12))[0]
    
    # # If flags < 4 exist, use those to construct mask, discard land class
    # if np.where(gdf.geolocation_qual < 4)[0].size != 0:
    #     mask = np.where(np.logical_and(gdf.classification > 1, gdf.geolocation_qual < 4))[0]
    # # If flags < 4 do not exist, construct mask with xovercal_suspect, discards land class
    # else:
    #     flags = np.logical_or.reduce([gdf.geolocation_qual == 64,
    #                                   gdf.geolocation_qual == 65,
    #                                   gdf.geolocation_qual == 66,
    #                                   gdf.geolocation_qual == 67])
    #     mask = np.where(np.logical_and(gdf.classification > 1, flags))[0]
    return mask

In [651]:
def makeGDF(pixc):
    # This function takes the pixel cloud, makes a GeoDataFrame,
    # renames columns as needed, projects the CRS, and returns
    # the GDF 
    
    # Create GDF
    gdf_PIXC = gpd.GeoDataFrame(ds_PIXC.classification[mask],
                        geometry=gpd.points_from_xy(ds_PIXC.longitude[mask],
                                                    ds_PIXC.latitude[mask]),
                        crs="EPSG:4326") # PIXC has no native CRS, setting same as River_SP

    gdf_PIXC.rename(columns={gdf_PIXC.columns[0]: 'klass'}, inplace=True)
    
    # Convert the crs to WGS 84 / Pseudo-Mercator
    gdf_PIXC = gdf_PIXC.to_crs(epsg=3857)
    return gdf_PIXC

In [705]:
folder = '/nas/cee-water/cjgleason/fiona/data/small_rivers/mar_2024_ver_c/'
season = 'leaf_on' # 'leaf_off'

# Label
on_off = 'Leaf on:'

In [706]:
files = listdir(join(folder, season))
files

['SWOT_L2_HR_PIXC_018_298_080L_20240720T131306_20240720T131317_PIC0_03.nc',
 'SWOT_L2_HR_PIXC_016_341_229R_20240610T090004_20240610T090015_PIC0_01.nc']

In [707]:
# Need to figure out how to open multiple xarrays with different
# dimensions, but for now:

# Open both PIXC for the given season and store in dict
d = {}

for i in range(2):
    print(i)
    # Read in xarray
    ds_PIXC = xr.open_mfdataset(paths=join(folder, season, files[i]),
                            group='pixel_cloud', engine='h5netcdf')
    if i == 0:
        # Make dict for legend labels
        flags = ds_PIXC.classification.flag_meanings.split() # extract each flag meaning
        codes = {idx:k for idx, k in enumerate(flags, start=1)}

    # Make mask
    mask = bitwiseMask(ds_PIXC)
    
    # Make PIXC
    gdf_PIXC = makeGDF(pixc=ds_PIXC)
    
    d[i] = gdf_PIXC

0
1


In [708]:
# Merge masked GeoDataFrames
gdf_PIXC = pd.concat([d[0], d[1]])

In [709]:
# gdf_PIXC

#### Sjoin

In [695]:
# Reaches for PTs
fort = [10000900041465, 10000900053916, 10000900041464, 10000900016682,
        10000900090995, 10000900090997, 10000900078861, 10000900004532,
        10000900004534, 10000900016914, 10000900091439, 10000900041902,
        10000900079115, 10000900029888] 
        
cushman = [10000900091444, 10000900091445, 10000900054636,
           10000900042171, 10000900005063,10000900067147, 10000900042169,
           10000900029899, 10000900079371, 10000900005062, 10000900017467,
           10000900054634, 10000900054635, 10000900017468, 10000900042170,
           10000900054639, 10000900017746]
# This is the beaver marsh: 10000900042169, 10000900029899, 10000900079371,
# 10000900005062, 10000900017467, 10000900054634, 10000900054635, 1000090001746 

In [710]:
# Get just the reaches of interest
reaches = basin[basin['NHDPlusID'].isin(cushman)]

# Label
river = 'cushman_brook'

In [711]:
# Buffer reach by 1/2 channel width
reaches['geometry'] = reaches['geometry'].buffer(distance=(reaches['WidthM']/2), cap_style='flat')

/work/pi_cjgleason_umass_edu/.conda/envs/narrowPIXC/lib/python3.12/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [712]:
# Join pixel cloud and buffered reaches 
inside = gpd.sjoin(gdf_PIXC, reaches, how='inner', predicate='within').rename(columns={'index_right': 'segment'})

In [713]:
# Which reaches have pixels?
list(set(inside.NHDPlusID) & set(reaches.NHDPlusID))

[10000900054636.0, 10000900091445.0, 10000900091444.0, 10000900054639.0]

#### Plotting

In [714]:
outpath = '/nas/cee-water/cjgleason/fiona/narrow_rivers_PIXC/figures_for_PTs/'

In [715]:
# Make color palette
# palette = {2: 'darkolivegreen', 3: 'slateblue', 4: 'steelblue', 5: 'hotpink',
#            6: 'saddlebrown', 7: 'darkslategray'}
palette = {2: 'darkkhaki', 3: 'cornflowerblue', 4: 'blue', 5: 'hotpink',
           6: 'lightsalmon', 7: 'lightblue'}

In [ ]:
# Plot the full set of reaches
# Get labels for plot
nums = list(np.unique(inside.klass).astype('int'))
labels = [codes[x] for x in nums]

# Make cmap
cmap = colors.ListedColormap([palette[x] for x in nums])

# Plot
fig, ax = plt.subplots(figsize=(15,15))

reaches.plot(color='w', alpha=0.2, ax=ax)
inside.plot(column=inside.klass, categorical=True, 
            markersize=10, cmap=cmap, legend=True,
            legend_kwds={'labels': labels,
                         'framealpha': 1, 
                         'title_fontsize': 'medium',
                         'loc': 'upper left'},
            ax=ax)

# Basemap
cx.add_basemap(ax, crs=gdf_PIXC.crs, source=cx.providers.CartoDB.DarkMatter, alpha=0.9)

# 
leg = ax.get_legend()
leg.set_title("Class")          
ax.title.set_text(on_off + '\n' + files[0] + ' and \n' \
                  + files[0] + '\nfor the ' + reaches.iloc[0, 1])
plt.savefig(fname = outpath + river +'_' + season + '_masked'+ '.png', bbox_inches='tight')

In [ ]:
# Plot each reach individually
count = 0

for plusid in reaches['NHDPlusID']:
    # Subset to reach, get vars
    reach = reaches[reaches['NHDPlusID'] == plusid]
    name = reach['GNIS_Name'].iloc[0]
    wd = str(np.round(reach['WidthM'].iloc[0], decimals=2))
    ln = str(np.round(reach['LengthKM'].iloc[0], decimals=2))
    
    points = inside[inside['NHDPlusID'] == plusid]
    
    # Get labels for plot
    nums = list(np.unique(points.klass).astype('int'))
    labels = [codes[x] for x in nums]

    # Make cmap
    cmap = colors.ListedColormap([palette[x] for x in nums])

    # Plot
    fig, ax = plt.subplots(figsize=(15,15))

    reach.plot(color='w', alpha=0.2, ax=ax)
    if points.size != 0:
        points.plot(column=points.klass, categorical=True,
                    # markersize=10,
                    cmap=cmap, legend=True,
                    legend_kwds={'labels': labels,
                                 'framealpha': 1, 
                                 'title_fontsize': 'medium',
                                 'loc': 'upper left'},
                    ax=ax)
        leg = ax.get_legend()
        leg.set_title("Class")          
    ax.title.set_text(on_off + ' ' + name + ', NHDPlusID ' + str(int(plusid)) + ', ' + wd + ' m wide, ' + ln + ' km long')

    # Basemap
    cx.add_basemap(ax, crs=gdf_PIXC.crs, source=cx.providers.CartoDB.DarkMatter,
                   # zoom=20, 
                   alpha=0.9)
    
    # Save fig
    plt.savefig(fname = outpath + river +'_' + season + '_masked_' + str(count) + '.png', bbox_inches='tight')    
    count += 1

In [68]:
# Get bounds of PIXC, construct polygon
bounds_PIXC = gdf_PIXC.total_bounds
poly = box(bounds_PIXC[0], bounds_PIXC[1], bounds_PIXC[2], bounds_PIXC[3])

In [69]:
crop_basin = basin.clip(poly)

In [70]:
bins = np.unique(crop_basin.Bin)
bins

array(['(0, 10]', '(10, 20]', '(150, 200]', '(20, 30]', '(200, 500]',
       '(30, 40]', '(40, 50]', '(50, 60]', '(60, 70]'], dtype=object)